In [54]:
import pandas as pd
import rdfpandas
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, OWL
from rdflib import Graph
import spacy
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import json

In [2]:
df = pd.read_csv("movies_metadata.csv")

/home/amj/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [56]:
df.shape

(45466, 24)

In [4]:
url = "https://api.dbpedia-spotlight.org/en/annotate"
headers = {'accept': 'application/json'}
def find_dbpedia_entry(title):
    parameters = {
    'text' : title
    }
    response = requests.get(url, params = parameters, headers = headers)
    return response.json()['Resources'][0]['@URI']

In [ ]:
find_dbpedia_entry('Toy Story')

In [5]:
for (idx, row) in df[:10].iterrows():
    try:
        print(find_dbpedia_entry(row['original_title']))
    except:
        continue

http://dbpedia.org/resource/Jumanji
http://dbpedia.org/resource/Grumpier_Old_Men
http://dbpedia.org/resource/Sabrina_Spellman


In [57]:
g = Graph()

In [58]:
# NS = Namespace('http://movies.org/')
NS = Namespace('#@')
MV = Namespace('https://www.imdb.com/')

In [59]:
g.bind('schema', SDO)
g.bind('owl', OWL)
g.bind('movie', MV)

In [60]:
for (idx, row) in df[:10000].iterrows():
    movie = NS[row['imdb_id']]
#     title = Literal(row['original_title'], lang = row['original_language'])
    title = Literal(row['original_title'])
    try:
        URI = URIRef(find_dbpedia_entry(row['original_title']))
    except KeyError:
        continue
    rel_date = row['release_date']
    release_date = Literal(rel_date, datatype=XSD['date'])
    
    g.add((movie, MV.title, title))
    g.add((movie, MV['releaseDate'], release_date))
    g.add((movie, OWL.sameAs, URI))

In [61]:
op = g.serialize(format='xml')
# print(op)

In [62]:
with open('movie.owl', 'w') as fo:
    fo.write(op)

In [63]:
len(g)

20715

# TILL HERE: FILE WRITTEN

# BELOW: DBPEDIA

In [2]:
g = Graph()
g.parse("movie.owl")

<Graph identifier=N78bedc29853c44f6919944409065e383 (<class 'rdflib.graph.Graph'>)>

In [3]:
def get_dbpedia_link(title):
    query = '''
                        PREFIX owl: <http://www.w3.org/2002/07/owl#>
                        PREFIX movie: <https://www.imdb.com/>

                        SELECT ?remote
                        WHERE {
                            ?movie movie:title "'''+title+'''" .
                            ?movie owl:sameAs ?remote .
                        }
                        '''
    result = g.query(query)
    for i in result:
        db_link = str(i[0])
        print(db_link)
        break
    return db_link

In [108]:
def get_director(title):
    db_link = get_dbpedia_link(title)
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?remote_value
            WHERE {
                    <'''+db_link+'''> dbp:director ?remote_value
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    print(results)
    try:
        return results['results']['bindings'][0]['remote_value']['value']
    except:
        print("Sorry, no results")

link = get_dbpedia_link("Before Sunrise")
director = get_director("Before Sunrise")
director


http://dbpedia.org/resource/Before_Sunrise
http://dbpedia.org/resource/Before_Sunrise
{'head': {'link': [], 'vars': ['remote_value']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'remote_value': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Richard_Linklater'}}]}}


'http://dbpedia.org/resource/Richard_Linklater'

In [92]:
link = get_dbpedia_link("A Kiss Before Dying")
get_director(link)

http://dbpedia.org/resource/A_Kiss_Before_Dying_(1991_film)


'http://dbpedia.org/resource/James_Dearden'

In [101]:
def get_movies(director):
#     try:
        query = '''
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX movie: <https://www.imdb.com/>

                SELECT ?budget
                WHERE {
                        ?movie dbo:director <'''+director+'''> .
                        ?movie dbo:budget ?budget
                    }
                '''   
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        return [record['budget']['value'] for record in results['results']['bindings']]
#     except:
        print("Sorry! no results")
#     return results

def get_avg_budget(director):
    result = get_movies(director)
    result_int = list()
    try:
        for i in result:
            if 'E' in i:
                b,e = i.split('E')
                num = float(b)*np.power(10,int(e))
                result_int.append(num)
            else:
                result_int.append(float(i))
        print("The average budget of "+director.split('/')[-1]+"'s movies is: Dollars", np.average(result_int))
        return np.average(result_int)
    except:
        print("Sorry! No results")

In [98]:
get_director("A Kiss Before Dying")

http://dbpedia.org/resource/A_Kiss_Before_Dying_(1991_film)


'http://dbpedia.org/resource/James_Dearden'

In [94]:
get_avg_budget('http://dbpedia.org/resource/James_Dearden')

The average budget of James_Dearden's movies is: Dollars 9540000.0


9540000.0

## GET AVERAGE BUDGET OF A MOVIE's DIRECTOR's ALL MOVIES

In [302]:
get_avg_budget(get_director(get_dbpedia_link("A Kiss Before Dying")))

http://dbpedia.org/resource/A_Kiss_Before_Dying_(1991_film)
The average budget of James_Dearden's movies is: Dollars 9540000.0


9540000.0

In [103]:
get_avg_budget(get_director("A Kiss Before Dying"))

http://dbpedia.org/resource/A_Kiss_Before_Dying_(1991_film)
The average budget of James_Dearden's movies is: Dollars 9540000.0


9540000.0

In [104]:
get_director("The Matrix")

http://dbpedia.org/resource/The_Matrix


'The Wachowskis'

In [105]:
get_director("A Kiss Before Dying")

http://dbpedia.org/resource/A_Kiss_Before_Dying_(1991_film)


'http://dbpedia.org/resource/James_Dearden'

In [102]:
get_avg_budget(get_director("The Matrix"))

http://dbpedia.org/resource/The_Matrix


QueryBadFormed: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 8: syntax error at '<' before 'The'\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\n                PREFIX owl: <http://www.w3.org/2002/07/owl#>\n                PREFIX movie: <https://www.imdb.com/>\n\n                SELECT ?budget\n                WHERE {\n                        ?movie dbo:director <The Wachowskis> .\n                        ?movie dbo:budget ?budget\n                    }\n                "

# GET ALL MOVIES WHERE THE COACTORS OF A MOVIE STARRED

In [8]:
def get_coactors(link):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?remote_value
            WHERE {
                    <'''+link+'''> dbo:starring ?remote_value .
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    try:
        if len(results['results']['bindings']) > 1:
            return results['results']['bindings']
    except:
        print("Sorry, no results")

In [131]:
def get_coactors_movies(actors):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?film
            WHERE {
                    ?film dbo:starring <'''+actors[0]+'''> .
                    ?film dbo:starring <'''+actors[1]+'''> .
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    try:
#         if len(results['results']['bindings']) > 1:
        return results['results']['bindings']
    except:
        print("Sorry, no results")

In [9]:
def get_movies(director):
    try:
        query = '''
                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                PREFIX movie: <https://www.imdb.com/>

                SELECT ?budget
                WHERE {
                        ?movie dbo:director <'''+director+'''> .
                        ?movie dbo:budget ?budget
                    }
                '''   
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        return [record['budget']['value'] for record in results['results']['bindings']]
    except:
        print("Sorry! no results")

In [10]:
link = get_dbpedia_link("The Bunker")

http://dbpedia.org/resource/The_Bunker_(1981_film)


In [11]:
actors = [record['remote_value']['value'] for record in get_coactors(link)]
actors

['http://dbpedia.org/resource/Richard_Jordan',
 'http://dbpedia.org/resource/Anthony_Hopkins',
 'http://dbpedia.org/resource/Susan_Blakely',
 'http://dbpedia.org/resource/Cliff_Gorman']

In [129]:
actors = [record['remote_value']['value'] for record in get_coactors(link)]
actors

['http://dbpedia.org/resource/Julie_Delpy',
 'http://dbpedia.org/resource/Ethan_Hawke']

In [12]:
[film['film']['value'].split('/')[-1] for film in get_coactors_movies(actors)]

['The_Bunker_(1981_film)']

In [133]:
[film['film']['value'].split('/')[-1] for film in get_coactors_movies(actors)]

['Before_Sunrise', 'Before_Sunset', 'Before_Midnight', 'Before_trilogy']

In [125]:
get_coactors_movies.__code__.co_names

('SPARQLWrapper',
 'setQuery',
 'setReturnFormat',
 'JSON',
 'query',
 'convert',
 'print')

In [127]:
import inspect
print(inspect.getsource(get_coactors_movies))

def get_coactors_movies(actors):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?film
            WHERE {
                    ?film dbo:starring <'''+actors[0]+'''> .
                    ?film dbo:starring <'''+actors[1]+'''> .
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    try:
#         if len(results['results']['bindings']) > 1:
        return results['results']['bindings']
    except:
        print("Sorry, no results")



In [117]:
dir(get_coactors_movies)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

# GET ALL RELATED MOVIES OF A MOVIE

In [13]:
link = get_dbpedia_link("The Matrix")

http://dbpedia.org/resource/The_Matrix


'http://dbpedia.org/resource/The_Matrix'

In [79]:
def get_related_movies(db_link):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?remote_value
            WHERE {
                    <'''+db_link+'''> dbo:wikiPageWikiLink ?remote_value
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    try:
        return results['results']['bindings']
    except:
        print("Sorry, no results")
        
def check_movie(results):
    for result in results:
        query = '''
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX movie: <https://www.imdb.com/>

        SELECT ?remote_value
        WHERE {
                <'''+result['remote_value']['value']+'''> rdf:type ?remote_value
            }
        '''
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        output = sparql.query().convert()
        for rv in output['results']['bindings']:
            if rv['remote_value']['value'] == 'http://dbpedia.org/ontology/Film':
                        query = '''
                                PREFIX owl: <http://www.w3.org/2002/07/owl#>
                                PREFIX movie: <https://www.imdb.com/>

                                SELECT ?remote_value
                                WHERE {
                                        <'''+result['remote_value']['value']+'''> dbp:name ?remote_value
                                    }
                                '''
                        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
                        sparql.setQuery(query)
                        sparql.setReturnFormat(JSON)
                        movie_name = sparql.query().convert()
                        print(movie_name['results']['bindings'][0]['remote_value']['value'])        

link = get_dbpedia_link("The Matrix")
rel_movies = get_related_movies(link)



http://dbpedia.org/resource/The_Matrix


In [137]:
def get_movie_name(link):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?remote_value
            WHERE {
                    <'''+link+'''> dbp:name ?remote_value
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    movie_name = sparql.query().convert()
    print(movie_name['results']['bindings'][0]['remote_value']['value']) 

In [313]:
def get_actor_name(link):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?remote_value
            WHERE {
                    <'''+link+'''> dbp:name ?remote_value
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    actor_name = sparql.query().convert()
    return actor_name['results']['bindings'][0]['remote_value']['value']

In [142]:
get_actor_name("http://dbpedia.org/resource/Keanu_Reeves")

Keanu Reeves


In [138]:
get_movie_name("http://dbpedia.org/resource/The_Matrix")

The Matrix


In [80]:
check_movie(rel_movies)

Marx Reloaded
The Matrix Reloaded
Tron: Legacy
The Matrix Revolutions
Assassins
Bound
Strange Days
The Animatrix
The Helix...Loaded
The Matrix: Resurrections
Tron
Night Watch
Crouching Tiger, Hidden Dragon
Daredevil
Dark City
Deuce Bigalow: Male Gigolo
Belinda McClory
Metropolis
Wanted
2001
{'remote_value': {'type': 'uri', 'value': 'http://dbpedia.org/resource/2001:_A_Space_Odyssey_(film)'}}
World on a Wire
Scary Movie
Awara Paagal Deewana
Hugo Weaving
Akira
Equilibrium
The Phantom Menace
Ghost in the Shell
Inception
JFK


IndexError: list index out of range

# GET YOUNGEST CREW MEMBER

In [260]:
def get_crew(link):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?dir ?dop ?comp ?music ?screen ?prod group_concat(?star; separator=",")
            WHERE {
                    <'''+link+'''> dbo:director ?dir .
                    <'''+link+'''> dbo:cinematography ?dop .
                    <'''+link+'''> dbo:musicComposer ?comp .
                    <'''+link+'''> dbp:music ?music .
                    <'''+link+'''> dbo:producer ?prod .
                    <'''+link+'''> dbo:starring ?star .
                }
            '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    crew = sparql.query().convert()
#     print(crew) 
    return crew

In [261]:
result = get_crew(get_dbpedia_link("The Matrix"))

http://dbpedia.org/resource/The_Matrix
{'head': {'link': [], 'vars': ['dir', 'dop', 'comp', 'music', 'screen', 'prod', 'callret-6']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'dir': {'type': 'uri', 'value': 'http://dbpedia.org/resource/The_Wachowskis'}, 'dop': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Bill_Pope'}, 'comp': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Don_Davis_(composer)'}, 'music': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Don_Davis_(composer)'}, 'prod': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Joel_Silver'}, 'callret-6': {'type': 'literal', 'value': 'http://dbpedia.org/resource/Keanu_Reeves,http://dbpedia.org/resource/Hugo_Weaving,http://dbpedia.org/resource/Laurence_Fishburne,http://dbpedia.org/resource/Joe_Pantoliano,http://dbpedia.org/resource/Carrie-Anne_Moss'}}]}}


In [224]:
l = [r['value'].split(',') for r in result['results']['bindings'][0].values()]
links = [y for x in l for y in x]

In [236]:
links

['http://dbpedia.org/resource/The_Wachowskis',
 'http://dbpedia.org/resource/Bill_Pope',
 'http://dbpedia.org/resource/Don_Davis_(composer)',
 'http://dbpedia.org/resource/Don_Davis_(composer)',
 'http://dbpedia.org/resource/Joel_Silver',
 'http://dbpedia.org/resource/Keanu_Reeves',
 'http://dbpedia.org/resource/Hugo_Weaving',
 'http://dbpedia.org/resource/Laurence_Fishburne',
 'http://dbpedia.org/resource/Joe_Pantoliano',
 'http://dbpedia.org/resource/Carrie-Anne_Moss']

In [258]:
def get_youngest(links):
    query = '''
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX movie: <https://www.imdb.com/>
        SELECT ?bd ?name WHERE
        {
        {SELECT ?bd ?name
        WHERE {
                <'''+links[0]+'''> dbo:birthDate ?bd .
                <'''+links[0]+'''> dbp:name ?name .
            }}'''
    if len(links) > 1:
        for link in links:
            query = query + '''UNION
                {SELECT ?bd ?name
            WHERE {
                    <'''+link+'''> dbo:birthDate ?bd .
                    <'''+link+'''> dbp:name ?name .
                }}
            '''
    query = query + '''}
        ORDER BY ASC(?bd)
    '''
        
#     query = '''
#         PREFIX owl: <http://www.w3.org/2002/07/owl#>
#         PREFIX movie: <https://www.imdb.com/>
#         SELECT ?bd ?name WHERE
#         {
#         {SELECT ?bd ?name
#         WHERE {
#                 <'''+links[2]+'''> dbo:birthDate ?bd .
#                 <'''+links[2]+'''> dbp:name ?name .
#             }}
#             UNION
#             {SELECT ?bd ?name
#         WHERE {
#                 <'''+links[1]+'''> dbo:birthDate ?bd .
#                 <'''+links[1]+'''> dbp:name ?name .
#             }}
#             UNION
#             {SELECT ?bd ?name
#         WHERE {
#                 <'''+links[3]+'''> dbo:birthDate ?bd .
#                 <'''+links[3]+'''> dbp:name ?name .
#             }}
#             UNION
#             {SELECT ?bd ?name
#         WHERE {
#                 <'''+links[4]+'''> dbo:birthDate ?bd .
#                 <'''+links[4]+'''> dbp:name ?name .
#             }}
#             }
#         ORDER BY ASC(?bd)
        
#         '''
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    movie_name = sparql.query().convert()
    print("The youngest crew member is", movie_name['results']['bindings'][0]['name']['value'],"born on", movie_name['results']['bindings'][0]['bd']['value']) 

In [259]:
get_youngest(links)

The youngest crew member is Bill Pope born on 1952-06-19


# GET LONGEST RUNNING TIME OF A MOVIE's TOP ACTOR

In [315]:
def get_longest_movie(title):
    link = get_dbpedia_link(title)
    actor = get_coactors(link)[0]['remote_value']['value']
    actor_name = get_actor_name(actor)
#     print(actor)
#     try:
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?runtime ?name
            WHERE {
                    ?movie dbo:starring <'''+actor+'''> .
                    ?movie dbo:runtime ?runtime .
                    ?movie dbp:name ?name .
                }
            ORDER BY DESC(?runtime)
            '''   
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    print("The longest movie of", actor_name, "is", results['results']['bindings'][0]['name']['value'], "running", float(results['results']['bindings'][0]['runtime']['value'])/3600,"hours")
#     except:
#         print("Sorry! no results")

In [316]:
get_longest_movie("The Matrix")

http://dbpedia.org/resource/The_Matrix
The longest movie of Keanu Reeves is The Devil's Advocate running 2.4 hours


In [265]:
get_coactors("http://dbpedia.org/resource/The_Matrix")

[{'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Keanu_Reeves'}},
 {'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Hugo_Weaving'}},
 {'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Laurence_Fishburne'}},
 {'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Joe_Pantoliano'}},
 {'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Carrie-Anne_Moss'}}]

# ROUGH

In [282]:
def get_movies(director):
    query = '''
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX movie: <https://www.imdb.com/>

            SELECT ?budget
            WHERE {
                    ?movie dbo:director <'''+director+'''> .
                    ?movie dbo:budget ?budget
                }
            '''   
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return [record['budget']['value'] for record in results['results']['bindings']]
#     return results

def get_avg_budget(director):
    result = get_movies(director)

    result_int = list()
    for i in result:
        if 'E' in i:
            b,e = i.split('E')
            num = float(b)*np.power(10,int(e))
            result_int.append(num)
        else:
            result_int.append(float(i))
    return np.average(result_int)

In [283]:
get_avg_budget(director)

9943937.5

In [234]:
director

'http://dbpedia.org/resource/Richard_Linklater'

In [185]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(query2)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [193]:
results['results']['bindings']

[{'remote_property': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'remote_value': {'type': 'uri',
   'value': 'http://www.w3.org/2002/07/owl#Thing'}},
 {'remote_property': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/class/yago/WikicatAmericanFilms'}},
 {'remote_property': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/class/yago/WikicatAmericanRemakesOfFrenchFilms'}},
 {'remote_property': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/class/yago/WikicatAmericanScienceFictionFilms'}},
 {'remote_property': {'type': 'uri',
   'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
  'remote_value': {'type': 'uri',
   'value': 'http://dbpedia.org/class/y

In [140]:
query1 = '''
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX movie: <https://www.imdb.com/>

SELECT ?remote
WHERE {
    ?movie movie:title "Twelve Monkeys" .
    ?movie owl:sameAs ?remote . 
    ?movie movie:releaseDate ?reldate
}
'''

In [184]:
query2 = '''
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX movie: <https://www.imdb.com/>

SELECT ?remote ?remote_property ?remote_value
WHERE {
        <'''+db_link+'''> ?remote_property ?remote_value
    }
'''

In [183]:
print(query2)


PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX movie: <https://www.imdb.com/>

SELECT ?remote ?remote_property ?remote_value
WHERE {
        <http://dbpedia.org/resource/12_Monkeys> ?remote_property ?remote_value
    }
LIMIT 3



In [137]:
query = """
    SELECT ?s
    WHERE {
        ?s a ?o .
    }
    LIMIT 3
    """

In [141]:
result = g.query(query1)

In [175]:
for i in result:
    db_link = str(i[0])
    break

In [176]:
db_link

'http://dbpedia.org/resource/12_Monkeys'

In [56]:
# x = '34'

# bob = NS[x]
# sample_uri = URIRef("www.facebook.com/ajktym94")
# release_date = Literal("1990-07-04", datatype=XSD['date'])

# print(g.serialize(format='xml'))